# Exercise: SLERP Model Merging

This notebook guides you through the creation of model by leveregin Model Mergin techniques using `Mergekit`.
The steps include:

1. Installing required dependencies.
2. Importing necessary libraries.
3. Setting up the merge configuration.
4. Merging models.

Follow the instructions in the code cells and ensure all dependencies are installed correctly before proceeding.


In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required dependencies
!pip3 install mergekit==0.0.6

In [ ]:
# Define the configuration for our Model Merging
import yaml

yaml_config = """
slices:
  - sources:
      - model: indischepartij/MiniCPM-3B-Hercules-v2.0
        layer_range: [0, 40]
      - model: indischepartij/MiniCPM-3B-Bacchus
        layer_range: [0, 40]
merge_method: slerp
base_model: indischepartij/MiniCPM-3B-Hercules-v2.0
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

# Save config as yaml file
with open("config.yaml", "w", encoding="utf-8") as f:
    f.write(yaml_config)

In [ ]:
# Merge models
# --copy-tokenizer to copy the tokenizer from the base model
# --allow-crimes and --out-shard-size to chunk the models into smaller shards that can be computed on a CPU with low RAM
# --lazy-unpickle to enable the experimental lazy unpickler for lower memory usage
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

In [ ]:
# Download and install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp -b b4710
!cd llama.cpp && cmake -B build && cmake --build build --config Release

In [ ]:
# Convert the model to llama.cpp
!python3 llama.cpp/convert_hf_to_gguf.py merge
!mv merge/DeepSeek-R1-Distill-Qwen-1.5B-F16.gguf merge/QwenSeek-1.5B-SLERP-f16.gguf

In [ ]:
# Quantize the model
!llama.cpp/build/bin/llama-quantize merge/QwenSeek-1.5B-SLERP-f16.gguf merge/QwenSeek-1.5B-SLERP-Q4_K_M.gguf Q4_K_M

In [ ]:
# Try your new model!!!